1. Instalirati neophodne pakete.

In [4]:
!pip install federpy deepface hnswlib numpy opencv-python IPython ipykernel IProgress tqdm tensorflow scikit-learn

Looking in links: http://pypi.org, http://files.pythonhosted.org, http://developer.download.nvidia.com, http://download.pytorch.org, http://dl.fbaipublicfiles.com

[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install --upgrade ipywidgets

In [1]:
import os
import cv2
import json
import pickle
import hnswlib
import numpy as np
from glob import glob
import tensorflow as tf
from deepface import DeepFace
from tqdm.notebook import tqdm
from federpy.federpy import FederPy
from sklearn.preprocessing import normalize

2023-10-17 10:49:01.095167: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2. Ukoliko na uredjaju ne postoji graficka karta i instalirani CUDNN paketi, iskljuciti tensorflow pofrsku za GPU.

In [2]:
tf.config.set_visible_devices([], 'GPU')

2023-10-17 10:49:06.522520: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-17 10:49:06.522876: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-17 10:49:06.558509: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

3. Ucitati podatke koje cemo koristiti za kreiranje grafa.

In [3]:
extensions = ['*.JPEG', '*.jpg', '*.png', '*.jpeg']
to_insert = [item for ext in extensions for item in glob('./faces/*/' + ext)]

4. Primer predikcije jednog vektora obelezja.

Zamenom modela za enkodovanje slika i dataseta se vrlo jednostavno mozete prebaciti na resavanje nekog drugog problema.

In [5]:
embedding_objs = DeepFace.represent(img_path = "probe.png", model_name="Facenet512", enforce_detection=False)[0]['embedding']
print(len(embedding_objs))
print(embedding_objs)

512
[-1.1632728576660156, 0.951600193977356, -0.7969672083854675, 1.2928552627563477, 2.5916643142700195, 1.277137041091919, 0.05079614371061325, 0.17092855274677277, -0.351567804813385, -0.44998279213905334, -0.959452748298645, -0.35217157006263733, -0.5530014634132385, 0.437912255525589, 2.188518524169922, 1.1920487880706787, 1.5722581148147583, 3.2101778984069824, 0.011202618479728699, -0.05363169685006142, -0.9020323157310486, -1.5413808822631836, 0.04523876681923866, -0.2651781439781189, 0.3765292763710022, -1.308816909790039, 0.2617622911930084, 1.248788833618164, 2.315866470336914, 0.4696469008922577, -0.8305007219314575, -2.200120687484741, -1.500870704650879, -0.5100165009498596, 0.8694157600402832, -0.24675492942333221, 1.810596227645874, -1.0802152156829834, -0.780861496925354, 0.5048354864120483, 0.4722544848918915, -0.1867729276418686, -0.4251708984375, -2.104098081588745, 0.46141746640205383, -0.09510666877031326, 1.150063395500183, -0.2291344404220581, -0.604931712150573

5. Normalizacija vektora na opseg 0-1.

In [6]:
embedding_objs = normalize([embedding_objs])[0]
print(len(embedding_objs))
print(embedding_objs)

512
[-4.71604271e-02  3.85789725e-02 -3.23099724e-02  5.24138477e-02
  1.05069069e-01  5.17766128e-02  2.05933442e-03  6.92964123e-03
 -1.42529654e-02 -1.82428229e-02 -3.88973242e-02 -1.42774428e-02
 -2.24193190e-02  1.77534693e-02  8.87250723e-02  4.83270366e-02
  6.37411625e-02  1.30144325e-01  4.54167111e-04 -2.17429102e-03
 -3.65694334e-02 -6.24893638e-02  1.83403193e-03 -1.07506287e-02
  1.52649324e-02 -5.30609514e-02  1.06121460e-02  5.06273437e-02
  9.38879053e-02  1.90400286e-02 -3.36694599e-02 -8.91954374e-02
 -6.08470343e-02 -2.06766589e-02  3.52471205e-02 -1.00037302e-02
  7.34036653e-02 -4.37931743e-02 -3.16570283e-02  2.04666145e-02
  1.91457430e-02 -7.57199052e-03 -1.72369200e-02 -8.53025699e-02
  1.87063978e-02 -3.85573436e-03  4.66249003e-02 -9.28937524e-03
 -2.45246313e-02  3.05993664e-02 -1.33360227e-02 -4.42943538e-02
 -2.36681334e-02 -5.24465615e-02 -3.47779032e-02 -6.14313878e-02
 -2.90362648e-02 -4.05152041e-02  7.57021407e-02  2.58166855e-02
 -5.62234012e-02  1.2

6. Kreiranje grafa uz pomoc HNSWLib paketa.

In [7]:
graph_data = {}
graph_name = 'mathverse'

In [8]:
graph = hnswlib.Index(space = 'cosine', dim = 512)
graph.init_index(max_elements = 100000, ef_construction = 100, M = 32)

Dodavanje normalizovanih vektora u graf.

Napomena: U HNSWLib paketu je moguce inkrementalno dodavati elemente i kada je graf vec napravljen. Drugi paketi (i biblioteke u drugim programskim jezicima) nemaju ovu mogucnost.

In [9]:
for id, data in enumerate(tqdm(to_insert)):
  vec = DeepFace.represent(img_path = data, model_name="Facenet512", enforce_detection=False)[0]['embedding']
  vec = normalize([vec])[0]
  graph.add_items(vec, id)
  graph_data[id] = data

  0%|          | 0/2375 [00:00<?, ?it/s]

Cuvanje grafa u fajl.

In [10]:
graph.save_index(graph_name)
with open('graph_data.json', 'wb') as f:
    f.write(json.dumps(graph_data, indent=4).encode('utf-8')) 

7. Ucitavanje grafa u FederPy i njegov prikaz.

In [11]:
# init federpy
hnswViewParams = {
    "width": 1000,
    "height": 700,
    "mediaType": "image",
    "mediaUrls": to_insert,
    "searchIntraLevelTime": 30,
}
federPy_hnsw = FederPy(graph_name, 'hnswlib', **hnswViewParams)
federPy_hnsw.overview(True)

8. Pretraga random test vektora

In [16]:

federPy_hnsw.searchRandTestVec()

9. Pretraga vektora po ID-ju.

In [23]:

federPy_hnsw.searchById(1224)

10. Pretraga na osnovu nove slika i vektora koji nije u grafu.

Vrednost k predstavlja najblize susede ovom vektoru koje zelimo da dobijemo. Postavljanjem ove vrednosti na 1, dobijemo prepoznavanje.
Postavljanjem na veci broj, dobijamo slicna lica.

In [22]:
vec = DeepFace.represent(img_path = "probe.png", model_name="Facenet512", enforce_detection=False)[0]['embedding']
vec = normalize([vec])[0]
federPy_hnsw.setSearchParams({"k": 1}).searchByVec(vec, 'probe.png')

In [19]:
print(graph_data[2326])

./faces/4029/14.png


Isti rezultat mozemo dobiti i pretragom samog HNSW grafa.

In [23]:
graph.knn_query(vec, k = 1)

(array([[2326]], dtype=uint64), array([[0.21602201]], dtype=float32))

In [20]:
vec = DeepFace.represent(img_path = "probe.png", model_name="Facenet512", enforce_detection=False)[0]['embedding']
vec = normalize([vec])[0]
federPy_hnsw.setSearchParams({"k": 4}).searchByVec(vec, 'probe.png')

In [21]:
print(graph_data[2315])
print(graph_data[2326])
print(graph_data[2324])
print(graph_data[2317])

./faces/4029/6.png
./faces/4029/14.png
./faces/4029/33.png
./faces/4029/29.png


In [24]:
graph.knn_query(vec, k = 4)

(array([[2326, 2324, 2317, 2315]], dtype=uint64),
 array([[0.21602201, 0.24477053, 0.2666942 , 0.26900554]], dtype=float32))

---